In [45]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
import pickle
import sys
from sys import exc_info
import ast

In [13]:
df=pd.read_csv('movies_metadata.csv')

C:\Users\Udhaya\AppData\Local\Temp\ipykernel_6736\2212646091.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('movies_metadata.csv')


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [15]:
df1=df[['id','genres','original_title', 'overview']]

In [16]:
from ast import literal_eval

features = ['genres']
for feature in features:
    df1[feature] = df1[feature].apply(literal_eval)

C:\Users\Udhaya\AppData\Local\Temp\ipykernel_6736\1711377990.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[feature] = df1[feature].apply(literal_eval)


In [17]:
def get_list(x):
    if isinstance(x, list):
        nm = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(nm) > 4:
            nm = nm[:4]
        return nm

    #Return empty list in case of missing/malformed data
    return []

In [18]:
df1['genres'][0]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [19]:
features = [ 'genres']
for feature in features:
    df1[feature] = df1[feature].apply(get_list)

C:\Users\Udhaya\AppData\Local\Temp\ipykernel_6736\3687065399.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[feature] = df1[feature].apply(get_list)


In [20]:
df1

,id,genres,original_title,overview
0,862,"[Animation, Comedy, Family]",Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,"[Adventure, Fantasy, Family]",Jumanji,When siblings Judy and Peter discover an encha...
2,15602,"[Romance, Comedy]",Grumpier Old Men,A family wedding reignites the ancient feud be...
3,31357,"[Comedy, Drama, Romance]",Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,11862,[Comedy],Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...,...,...
45461,439050,"[Drama, Family]",رگ خواب,Rising and falling between a man and woman.
45462,111109,[Drama],Siglo ng Pagluluwal,An artist struggles to finish his work while a...
45463,67758,"[Action, Drama, Thriller]",Betrayal,"When one of her hits goes wrong, a professiona..."
45464,227506,[],Satana likuyushchiy,"In a small town live two brothers, one a minis..."


In [21]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              45466 non-null  object
 1   genres          45466 non-null  object
 2   original_title  45466 non-null  object
 3   overview        44512 non-null  object
dtypes: object(4)
memory usage: 1.4+ MB


In [27]:
df1['tags'] = df1['overview']+df1['genres']

C:\Users\Udhaya\AppData\Local\Temp\ipykernel_6736\1648271142.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['tags'] = df1['overview']+df1['genres']


In [25]:
# Define a function to convert list to string
def list_to_string(lst):
    return ', '.join(lst)

# Apply the function to the 'genres' column
df1['genres'] = df1['genres'].apply(list_to_string)


C:\Users\Udhaya\AppData\Local\Temp\ipykernel_6736\3439144492.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['genres'] = df1['genres'].apply(list_to_string)


In [28]:
df1

,id,genres,original_title,overview,tags
0,862,"Animation, Comedy, Family",Toy Story,"Led by Woody, Andy's toys live happily in his ...","Led by Woody, Andy's toys live happily in his ..."
1,8844,"Adventure, Fantasy, Family",Jumanji,When siblings Judy and Peter discover an encha...,When siblings Judy and Peter discover an encha...
2,15602,"Romance, Comedy",Grumpier Old Men,A family wedding reignites the ancient feud be...,A family wedding reignites the ancient feud be...
3,31357,"Comedy, Drama, Romance",Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","Cheated on, mistreated and stepped on, the wom..."
4,11862,Comedy,Father of the Bride Part II,Just when George Banks has recovered from his ...,Just when George Banks has recovered from his ...
...,...,...,...,...,...
45461,439050,"Drama, Family",رگ خواب,Rising and falling between a man and woman.,Rising and falling between a man and woman.Dra...
45462,111109,Drama,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,An artist struggles to finish his work while a...
45463,67758,"Action, Drama, Thriller",Betrayal,"When one of her hits goes wrong, a professiona...","When one of her hits goes wrong, a professiona..."
45464,227506,,Satana likuyushchiy,"In a small town live two brothers, one a minis...","In a small town live two brothers, one a minis..."


In [32]:
df2=df1.drop(columns=['genres','overview'])

In [33]:
df2

,id,original_title,tags
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...,...
45461,439050,رگ خواب,Rising and falling between a man and woman.Dra...
45462,111109,Siglo ng Pagluluwal,An artist struggles to finish his work while a...
45463,67758,Betrayal,"When one of her hits goes wrong, a professiona..."
45464,227506,Satana likuyushchiy,"In a small town live two brothers, one a minis..."


In [46]:
df3 = df2.sample(n=10000, random_state=42)

In [47]:
from sklearn.feature_extraction.text import CountVectorizer

In [48]:
cv=CountVectorizer(max_features=10000, stop_words='english')

In [49]:
cv

CountVectorizer(max_features=10000, stop_words='english')

In [51]:
vector=cv.fit_transform(df3['tags'].values.astype('U')).toarray()

In [52]:
vector.shape

(10000, 10000)

In [53]:
from sklearn.metrics.pairwise import cosine_similarity

In [54]:
similarity=cosine_similarity(vector)

In [55]:
similarity

array([[1.        , 0.07254763, 0.10259784, ..., 0.07352146, 0.06900656,
        0.02067246],
       [0.07254763, 1.        , 0.18608073, ..., 0.18668326, 0.05006262,
        0.10498163],
       [0.10259784, 0.18608073, 1.        , ..., 0.        , 0.07079923,
        0.08483797],
       ...,
       [0.07352146, 0.18668326, 0.        , ..., 1.        , 0.        ,
        0.03039738],
       [0.06900656, 0.05006262, 0.07079923, ..., 0.        , 1.        ,
        0.01426535],
       [0.02067246, 0.10498163, 0.08483797, ..., 0.03039738, 0.01426535,
        1.        ]])

In [60]:
df3[df3['original_title']=="Jumanji"].index[0]

1

In [62]:
distance = sorted(list(enumerate(similarity[2])), reverse=True, key=lambda vector:vector[1])
for i in distance[0:5]:
    print(df3.iloc[i[0]].original_title)

Bad Lieutenant
The Phenix City Story
Heli
Stella Does Tricks
Dead Man Down


In [66]:
def recommand(movies):
    index=df3[df3['original_title']==movies].index[0]
    distance = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector:vector[1])
    for i in distance[0:5]:
        print(df3.iloc[i[0]].original_title)

In [70]:
recommand('Up the Sandbox')

Beaches
Coluche, l'histoire d'un mec
Pani z przedszkola
Nisos 2: To kynigi tou hamenou thisavrou
Slaves of New York


In [68]:
df3

,id,original_title,tags
43526,411405,Small Crimes,"A disgraced former cop, fresh off a six-year p..."
6383,42492,Up the Sandbox,"A young wife and mother, bored with day-to-day..."
3154,12143,Bad Lieutenant,"While investigating a young nun's rape, a corr..."
10146,9976,Satan's Little Helper,A naïve young boy unknowingly becomes the pawn...
9531,46761,Sitcom,The adventures of an upper-class suburban fami...
...,...,...,...
43661,387399,We Go On,WE GO ON tells the story of one man's quest to...
19067,66956,Les amitiés particulières,A tale of the tender relationship between a tw...
41958,205236,El taxi de los conflictos,Musical. Thaddeus (Juanjo Menéndez) is a taxi ...
33771,70131,열혈남아,Jae-Mun is a seasoned veteran of the gang. Alt...


In [71]:

import pickle

In [72]:
pickle.dump(df2, open('movies_list.pkl', 'wb'))

In [73]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))